In [ ]:
!pip install requests numpy langchain langchain_huggingface langchain_community faiss openai

In [1]:
# 🔹 Import necessary libraries
import os
import requests
import numpy as np
from typing import List
from langchain_core.language_models import LLM
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import (
    TextLoader,
    PDFMinerLoader,
    CSVLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredPowerPointLoader,
)
from langchain.text_splitter import CharacterTextSplitter
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore


In [2]:
# 🔹 Custom LLM class to call Ollama locally
class LocalOllamaLLM(LLM):
    model_name: str = "llama3.2:3b"
    url: str = "http://localhost:11434/api/generate"

    def _call(self, prompt: str, stop: List[str] = None) -> str:
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }
        headers = {"Content-Type": "application/json"}

        try:
            response = requests.post(self.url, json=payload, headers=headers, timeout=120)
            response.raise_for_status()
            result = response.json()
            return result.get("response", "No response from AI.")
        except requests.RequestException as e:
            return f"Error: {str(e)}"

    @property
    def _llm_type(self) -> str:
        return "ollama"


In [3]:
# 🔹 Load and split all supported documents
def load_documents(directory="/home/remlab/ps-bpt/PythonAIRAG/data"):
    """
    Load and split supported documents (.txt, .pdf, .csv, .docx, .pptx) into chunks.
    """
    all_docs = []
    splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

    for file in os.listdir(directory):
        path = os.path.join(directory, file)

        try:
            if file.endswith(".txt"):
                loader = TextLoader(path)
            elif file.endswith(".pdf"):
                loader = PDFMinerLoader(path)
            elif file.endswith(".csv"):
                loader = CSVLoader(file_path=path)
            elif file.endswith(".docx") or file.endswith(".doc"):
                loader = UnstructuredWordDocumentLoader(path)
            elif file.endswith(".pptx"):
                loader = UnstructuredPowerPointLoader(path)
            else:
                print(f"❌ Skipped unsupported file: {file}")
                continue

            docs = loader.load()
            chunks = splitter.split_documents(docs)
            all_docs.extend(chunks)
            print(f"✅ Loaded and split: {file}")

        except Exception as e:
            print(f"⚠️ Failed to process {file}: {e}")

    return all_docs


In [4]:
# 🔹 Main RAG pipeline
def main():
    # Step 1: Load and split all documents
    documents = load_documents()

    # Step 2: Initialize embedding model
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Step 3: Embed the document chunks
    document_embeddings = [embedding_model.embed_query(doc.page_content) for doc in documents]
    dim = len(document_embeddings[0])
    index = faiss.IndexFlatL2(dim)

    # Step 4: Convert to NumPy array and add to FAISS index
    embeddings_np = np.array(document_embeddings).astype('float32')
    index.add(embeddings_np)

    # Step 5: Create in-memory docstore for mapping results
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
    index_to_docstore_id = {i: str(i) for i in range(len(documents))}

    # Step 6: Set up the FAISS vector store
    vector_db = FAISS(
        embedding_function=embedding_model,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id
    )

    # Step 7: Load the local LLM (Ollama)
    llm = LocalOllamaLLM()

    # Step 8: Build the RetrievalQA chain
    rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vector_db.as_retriever())

    # Step 9: Ask a question (from documents)
    question = "Summarize the Chewy machine details."
    result = rag_chain.invoke({"query": question})

    # Step 10: Print the answer
    print("\n📌 Question:", question)
    print("🤖 Answer:", result["result"])


In [5]:
# 🔹 Run the main function to start the process
if __name__ == "__main__":
    main()


✅ Loaded and split: DCA.pdf
✅ Loaded and split: lm-302-04-s2-2TB-EB0-2025-04-17.csv
✅ Loaded and split: ai.txt


Created a chunk of size 5594, which is longer than the specified 1000
Created a chunk of size 5762, which is longer than the specified 1000
Created a chunk of size 4254, which is longer than the specified 1000
Created a chunk of size 1117, which is longer than the specified 1000
Created a chunk of size 1349, which is longer than the specified 1000


✅ Loaded and split: SPI-winbond.pdf


/home/remlab/ps-bpt/PythonAIRAG/.venv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange



📌 Question: Summarize the Chewy machine details.
🤖 Answer: I can summarize the Chewy machine details from the provided context.

The Chewy machine has a unique address of 10.74.133.95. It is not attached to anything and does not have any additional information listed next to it.

However, there are other entries related to the Chewy machine, including:

- vc-ps-chewy-13: connected to GHS V4 with IP address 10.74.135.35.
- vc-ps-chewy-14: has a drive size of 16TB and is temporarily swapped with another drive on chewy-06.

The exact details about the Chewy machine's capabilities, specifications, or connections are not explicitly stated in the provided context.
